In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [4]:
df = pd.read_csv("../Data_Preprocessing/df_nnz_Km_clean.csv")

Разделяю данные на обучающую, тестовую и валидационную выборки. 

In [5]:
# Train Test Split
features = df.drop('Km', axis=1)
labels = df['Km']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

Вывожу на экран доли данных в каждой выборке относительно исходных данных.

In [7]:
for data in [y_train, y_val, y_test]:
  print(round(len(data) / len(labels), 2))

0.6
0.2
0.2


Сохраняю данные в формате CSV. Он сохраняет обучающие, валидационные и тестовые данные, а также соответствующие им метки классов.

In [8]:
# Save the data
X_train.to_csv('../Data_ML/train_features.csv', index=False)
X_val.to_csv('../Data_ML/val_features.csv', index=False)
X_test.to_csv('../Data_ML/test_features.csv', index=False)

y_train.to_csv('../Data_ML/train_labels.csv', index=False)
y_val.to_csv('../Data_ML/val_labels.csv', index=False)
y_test.to_csv('../Data_ML/test_labels.csv', index=False)